# Useful links

- [ ] https://cran.r-project.org/web/packages/afex/vignettes/introduction-mixed-models.pdf
- [ ] https://cran.r-project.org/web/packages/afex/afex.pdf
- [ ] https://cran.r-project.org/web/packages/afex/index.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_analysing_accuracy_data.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_anova_example.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_mixed_example.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/comparisons.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/messy-data.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/interactions.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/basics.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_mixed_example.html

- [ ] https://osf.io/nfwx7/

- [ ] https://stats.stackexchange.com/questions/13166/rs-lmer-cheat-sheet

# Setup

## Imports

In [1]:
library("dplyr")
library("tidyr")
library("afex")     # provides mixed() and attaches lme4 automatically.
library("emmeans")  # follow-up tests.
library("multcomp") # advanced control for multiple testing/Type 1 errors.
library("ggplot2")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: lme4

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4

************
Welcome to afex. For support visit: http://afex.singmann.science/

- Functions for ANOVAs: aov_car(), aov_ez(), and aov_4()
- Methods for calculating p-values with mixed(): 'KR', 'S', 'LRT', and 'PB'
- 'afex_aov' and 'mixed' objects can be passed to emmeans() for follow-up tests
- NEWS: library('emmeans') now needs to be called explicitly!
- Get and set global pac

In [2]:
library("car")
require("lattice")
require("ez")

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


Loading required package: lattice

Loading required package: ez



## Settings

In [3]:
theme_set(theme_bw(base_size = 15) +
            theme(legend.position="bottom",
                  panel.grid.major.x = element_blank()))

In [4]:
options(width=120)

# Create output directory

In [5]:
dir.create("data/t1007",showWarnings = FALSE,recursive = TRUE)

## Handy functions

### Function for logging information BEFORE execution

In [6]:
logging0 <- function(ifName,infoStr=""){
    t0 <- Sys.time()
    s0 <- gsub("\\s","_",t0)
    s1 <- gsub("^.","",tempfile("","",""))
    s2 <- gsub("/t1006/","/t1007/",ifName)
    ofName <- paste(paste(s2,"",s0,"",s1,infoStr,"_nrow",nrow(df3),sep="_"),".RData",sep="")
    lfName = paste(ofName ,".LOG.txt" ,sep="" ,collapse = "")
    log_msg <- toString(paste(
        paste("logs0" ,toString(lfName)     ,sep=": " ,collapse = "")
      , paste("time0" ,toString(Sys.time()) ,sep=": " ,collapse = "")
      , paste("nrows" ,toString(nrow(df3))  ,sep=": " ,collapse = "")
      , paste("ncols" ,toString(ncol(df3))  ,sep=": " ,collapse = "")
      , paste("vList" ,toString(ls())       ,sep=": " ,collapse = "")
      , paste("oSave" ,toString(ofName)     ,sep=": " ,collapse = "")        
      , "\n"
      , sep="\n"))
    cat(log_msg)
    write(paste(log_msg) ,file=lfName ,append=TRUE)
    return(list("ofName"=ofName,"lfName"=lfName,"t0"=t0))
}

### Function for logging information AFTER execution

In [7]:
logging1 <- function(lfName,t0){
    t1 <- Sys.time()
    d1 <- t1-t0
    log_msg <- toString(paste(
        paste("time1" ,toString(Sys.time()) ,sep=": " ,collapse = "")
      , paste("diff1" ,capture.output(d1)   ,sep=": " ,collapse = "")
      , "\n"
      , sep="\n"))
    cat(log_msg)
    write(paste(log_msg) ,file=lfName ,append=TRUE)
    return(list("d1" = d1))
}

# Load data

Depending on how much data you want to include select a file to be loaded (either by executing relevant cell or by clicking ↑ or ↓ above to move cells up or down, the last executed cell containing `ifName` defines the file to be loaded).

In [19]:
ifName="data/t1006/bigDF__df3_subsamp_e3x3p0.RData" #    3000

In [20]:
ifName="data/t1006/bigDF__df3_subsamp_e3x6p0.RData" #    6000

In [21]:
ifName="data/t1006/bigDF__df3_subsamp_e3x8p0.RData" #    8000

In [22]:
ifName="data/t1006/bigDF__df3_subsamp_e2x1p0.RData" #     100

In [23]:
ifName="data/t1006/bigDF__df3_subsamp_e3x1p0.RData" #    1000

In [24]:
ifName="data/t1006/bigDF__df3_subsamp_e3x2p0.RData" #    2000

In [25]:
ifName="data/t1006/bigDF__df3_subsamp_e3x4p0.RData" #    4000

In [26]:
ifName="data/t1006/bigDF__df3_subsamp_e2x5p0.RData" #     500

In [27]:
ifName="data/t1006/bigDF__df3_subsamp_e4x1p0.RData" #   10000

In [28]:
ifName="data/t1006/bigDF__df3_fullset_e6x1p5.RData" # 1505108

In [33]:
load(file=ifName)

# Quick look at data

In [40]:
str(df3)

'data.frame':	752554 obs. of  28 variables:
 $ LBL              : Factor w/ 4 levels "metoo_ctrl","metoo_orig",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ wordCnt          : num  11 7 11 27 30 26 8 12 12 23 ...
 $ vb01Cnt          : num  0 0 0 2 0 0 1 0 1 0 ...
 $ nn01Cnt          : num  1 2 0 2 3 4 1 4 1 4 ...
 $ ag01Cnt          : num  0 0 0 0 1 1 0 0 1 2 ...
 $ ag02Cnt          : num  0 0 0 0 1 0 0 0 1 0 ...
 $ vb01Prc          : num  0 0 0 0.0741 0 ...
 $ nn01Prc          : num  0.0909 0.2857 0 0.0741 0.1 ...
 $ ag01Prc          : num  0 0 0 0 0.0333 ...
 $ ag02Prc          : num  0 0 0 0 0.0333 ...
 $ ID               : chr  "919714354467045376" "919714354479722496" "919714354500653056" "919714354542600192" ...
 $ tweet_url        : chr  "/IamJonesy3/status/919714354467045376" "/CCmee_/status/919714354479722496" "/SarahJaneVlogs/status/919714354500653057" "/queentyreen/status/919714354542600195" ...
 $ user_id          : chr  "588400632" "242582835" "29298715" "3290716455" ...
 $ username     

In [41]:
summary(df3)

             LBL            wordCnt          vb01Cnt           nn01Cnt          ag01Cnt           ag02Cnt      
 metoo_ctrl    :369527   Min.   :  1.00   Min.   : 0.0000   Min.   : 0.000   Min.   : 0.0000   Min.   :0.0000  
 metoo_orig    :306511   1st Qu.:  9.00   1st Qu.: 0.0000   1st Qu.: 2.000   1st Qu.: 0.0000   1st Qu.:0.0000  
 sexstrike_ctrl: 44449   Median : 17.00   Median : 0.0000   Median : 3.000   Median : 0.0000   Median :0.0000  
 sexstrike_orig: 32067   Mean   : 16.96   Mean   : 0.7442   Mean   : 3.498   Mean   : 0.6333   Mean   :0.2349  
                         3rd Qu.: 24.00   3rd Qu.: 1.0000   3rd Qu.: 5.000   3rd Qu.: 1.0000   3rd Qu.:0.0000  
                         Max.   :106.00   Max.   :17.0000   Max.   :43.000   Max.   :21.0000   Max.   :7.0000  
    vb01Prc           nn01Prc          ag01Prc           ag02Prc             ID             tweet_url        
 Min.   :0.00000   Min.   :0.0000   Min.   :0.00000   Min.   :0.00000   Length:752554      Length:752554  

In [43]:
df3 %>%
    group_by(CAMP,TYPE) %>%
    summarise(n = n(), .groups = "drop") 

ERROR: Error: Must group by variables found in `.data`.
* Column `CAMP` is not found.
* Column `TYPE` is not found.


# Model definition

| variable/factor type        | variable name | symbol |
|:----------------------------|:--------------|--------|
| observation unit (tweet_id) | ID            | u1     |
| outcome (dependent)         | MEAS_VALUE    | d1     |
| between tweet               | CAMP          | b1     |
| between tweet               | TYPE          | b2     |
| within tweet                | MEAS_LEVEL    | w1     |

## Formula 0

In [22]:
# by-ID random intercepts and by-ID random slopes for MEAS_LEVEL
FORMULA <- MEAS_VALUE~CAMP*TYPE*MEAS_LEVEL+(MEAS_LEVEL||ID)

In [23]:
# by-ID random intercepts
FORMULA <- MEAS_VALUE~CAMP*TYPE*MEAS_LEVEL+(1|ID) 

In [1]:
# CONTROL <- lmerControl(
#     check.nobs.vs.nlev  <- "ignore",
#     check.nobs.vs.rankZ <- "ignore",
#     check.nobs.vs.nRE   <- "ignore"
# )

In [24]:
CONTROL <- lmerControl(optCtrl=list(maxfun=1e7))

In [25]:
CONTROL <- lmerControl(optCtrl=list(maxfun=1e6))

In [26]:
METHOD <- "LRT" # calculates p-vals via the likelihood ratio tests (use if random factors have > 50 levels)

In [27]:
METHOD <- "KR"  # Kenward-Roger approximation for degrees of freedom

In [28]:
METHOD <- "S"   # Satterthwaite approximation for degrees of freedom

In [29]:
log0 <- logging0(ifName,infoStr="_AFEX_MIXED_formula0")
m0.0 <- mixed(
    formula   = FORMULA,
    data      = df3,
    control   = CONTROL,
    method    = METHOD,
    expand_re = TRUE
)
save(m0.0,file=log0$ofName)
log1 <- logging1(log0$lfName,log0$t0)

logs0: data/t1007/bigDF__df3_fullset_e6x1p5.RData__2020-09-30_15:01:13__275382c4207__AFEX_MIXED_formula0__nrow_1505108.RData.LOG.txt
time0: 2020-09-30 15:01:13
nrows: 1505108
ncols: 5
vList: ifName, infoStr, lfName, ofName, s0, s1, s2, t0
oSave: data/t1007/bigDF__df3_fullset_e6x1p5.RData__2020-09-30_15:01:13__275382c4207__AFEX_MIXED_formula0__nrow_1505108.RData



Contrasts set to contr.sum for the following variables: CAMP, TYPE, MEAS_LEVEL, ID



Fitting one lmer() model. [DONE]
Calculating p-values. [DONE]
time1: 2020-09-30 15:03:34
diff1: Time difference of 2.343031 mins



In [30]:
m0.0

Mixed Model Anova Table (Type 3 tests, S-method)

Model: MEAS_VALUE ~ CAMP * TYPE * MEAS_LEVEL + (1 | ID)
Data: df3
                Effect           df           F p.value
1                 CAMP 1, 752550.48 1231.90 ***   <.001
2                 TYPE 1, 752550.48   70.57 ***   <.001
3           MEAS_LEVEL 1, 752549.90 1208.29 ***   <.001
4            CAMP:TYPE 1, 752550.48  359.91 ***   <.001
5      CAMP:MEAS_LEVEL 1, 752549.90   37.90 ***   <.001
6      TYPE:MEAS_LEVEL 1, 752549.90        1.25    .263
7 CAMP:TYPE:MEAS_LEVEL 1, 752549.90   54.89 ***   <.001
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘+’ 0.1 ‘ ’ 1

## Formula 1

In [34]:
# by-ID random intercepts and by-ID random slopes for MEAS_LEVEL
FORMULA <- MEAS_VALUE~CAMP*TYPE*MEAS_LEVEL+(MEAS_LEVEL||ID)


# TODO: FIXME: CHECK THIS OUT !!!
CONTROL <- lmerControl(optCtrl=list(maxfun=1e9))

In [33]:
log0 <- logging0(ifName,infoStr="_AFEX_MIXED_formula1")
m0.1 <- mixed(
    formula   = FORMULA,
    data      = df3,
    control   = CONTROL,
    method    = METHOD,
    expand_re = TRUE
)
save(m0.1,file=log0$ofName)
log1 <- logging1(log0$lfName,log0$t0)

logs0: data/t1007/bigDF__df3_fullset_e6x1p5.RData__2020-09-30_15:05:31__27523a8a6bb__AFEX_MIXED_formula1__nrow_1505108.RData.LOG.txt
time0: 2020-09-30 15:05:31
nrows: 1505108
ncols: 5
vList: ifName, infoStr, lfName, ofName, s0, s1, s2, t0
oSave: data/t1007/bigDF__df3_fullset_e6x1p5.RData__2020-09-30_15:05:31__27523a8a6bb__AFEX_MIXED_formula1__nrow_1505108.RData



Contrasts set to contr.sum for the following variables: CAMP, TYPE, MEAS_LEVEL, ID



Fitting one lmer() model. 

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00236551 (tol = 0.002, component 1)”


[DONE]
Calculating p-values. [DONE]
time1: 2020-09-30 15:09:05
diff1: Time difference of 3.555083 mins



In [33]:
m0.1

Warning message:
“lme4 reported (at least) the following warnings for 'full':
  * Model failed to converge with max|grad| = 0.00236551 (tol = 0.002, component 1)”


Mixed Model Anova Table (Type 3 tests, S-method)

Model: MEAS_VALUE ~ CAMP * TYPE * MEAS_LEVEL + (MEAS_LEVEL || ID)
Data: df3
                Effect           df           F p.value
1                 CAMP 1, 752550.58 1231.90 ***   <.001
2                 TYPE 1, 752550.58   70.57 ***   <.001
3           MEAS_LEVEL 1, 752549.55 1208.29 ***   <.001
4            CAMP:TYPE 1, 752550.58  359.91 ***   <.001
5      CAMP:MEAS_LEVEL 1, 752549.55   37.90 ***   <.001
6      TYPE:MEAS_LEVEL 1, 752549.55        1.25    .263
7 CAMP:TYPE:MEAS_LEVEL 1, 752549.55   54.89 ***   <.001
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘+’ 0.1 ‘ ’ 1

# Quick look at models

In [34]:
m0.0

Mixed Model Anova Table (Type 3 tests, S-method)

Model: MEAS_VALUE ~ CAMP * TYPE * MEAS_LEVEL + (1 | ID)
Data: df3
                Effect           df           F p.value
1                 CAMP 1, 752550.48 1231.90 ***   <.001
2                 TYPE 1, 752550.48   70.57 ***   <.001
3           MEAS_LEVEL 1, 752549.90 1208.29 ***   <.001
4            CAMP:TYPE 1, 752550.48  359.91 ***   <.001
5      CAMP:MEAS_LEVEL 1, 752549.90   37.90 ***   <.001
6      TYPE:MEAS_LEVEL 1, 752549.90        1.25    .263
7 CAMP:TYPE:MEAS_LEVEL 1, 752549.90   54.89 ***   <.001
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘+’ 0.1 ‘ ’ 1

In [35]:
m0.1

Warning message:
“lme4 reported (at least) the following warnings for 'full':
  * Model failed to converge with max|grad| = 0.00236551 (tol = 0.002, component 1)”


Mixed Model Anova Table (Type 3 tests, S-method)

Model: MEAS_VALUE ~ CAMP * TYPE * MEAS_LEVEL + (MEAS_LEVEL || ID)
Data: df3
                Effect           df           F p.value
1                 CAMP 1, 752550.58 1231.90 ***   <.001
2                 TYPE 1, 752550.58   70.57 ***   <.001
3           MEAS_LEVEL 1, 752549.55 1208.29 ***   <.001
4            CAMP:TYPE 1, 752550.58  359.91 ***   <.001
5      CAMP:MEAS_LEVEL 1, 752549.55   37.90 ***   <.001
6      TYPE:MEAS_LEVEL 1, 752549.55        1.25    .263
7 CAMP:TYPE:MEAS_LEVEL 1, 752549.55   54.89 ***   <.001
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘+’ 0.1 ‘ ’ 1

## Select model for further inspection

In [36]:
m0 <- m0.1

In [37]:
m0 <- m0.0

In [38]:
summary(m0)$varcor

 Groups   Name        Std.Dev.
 ID       (Intercept) 0.035764
 Residual             0.059575

In [39]:
m3.0 <- emmeans(m0, c("CAMP", "TYPE", "MEAS_LEVEL"))
m3.0

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 1505108' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 1505108)' or larger];
but be warned that this may result in large computation time and memory use.

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 1505108' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 1505108)' or larger];
but be warned that this may result in large computation time and memory use.



 CAMP      TYPE MEAS_LEVEL  emmean        SE  df asymp.LCL asymp.UCL
 metoo     ctrl vb01Prc    0.04612 0.0001143 Inf   0.04590   0.04635
 sexstrike ctrl vb01Prc    0.04977 0.0003296 Inf   0.04912   0.05041
 metoo     orig vb01Prc    0.04167 0.0001255 Inf   0.04142   0.04191
 sexstrike orig vb01Prc    0.05100 0.0003880 Inf   0.05024   0.05176
 metoo     ctrl ag01Prc    0.04085 0.0001143 Inf   0.04063   0.04108
 sexstrike ctrl ag01Prc    0.04409 0.0003296 Inf   0.04344   0.04474
 metoo     orig ag01Prc    0.03362 0.0001255 Inf   0.03338   0.03387
 sexstrike orig ag01Prc    0.04737 0.0003880 Inf   0.04661   0.04813

Degrees-of-freedom method: asymptotic 
Confidence level used: 0.95 

In [40]:
pairs(m3.0)

 contrast                                         estimate       SE  df z.ratio p.value
 metoo ctrl vb01Prc - sexstrike ctrl vb01Prc     -0.003645 0.000349 Inf -10.450 <.0001 
 metoo ctrl vb01Prc - metoo orig vb01Prc          0.004454 0.000170 Inf  26.237 <.0001 
 metoo ctrl vb01Prc - sexstrike orig vb01Prc     -0.004878 0.000405 Inf -12.058 <.0001 
 metoo ctrl vb01Prc - metoo ctrl ag01Prc          0.005269 0.000139 Inf  38.017 <.0001 
 metoo ctrl vb01Prc - sexstrike ctrl ag01Prc      0.002031 0.000349 Inf   5.823 <.0001 
 metoo ctrl vb01Prc - metoo orig ag01Prc          0.012497 0.000170 Inf  73.618 <.0001 
 metoo ctrl vb01Prc - sexstrike orig ag01Prc     -0.001248 0.000405 Inf  -3.084 0.0427 
 sexstrike ctrl vb01Prc - metoo orig vb01Prc      0.008099 0.000353 Inf  22.965 <.0001 
 sexstrike ctrl vb01Prc - sexstrike orig vb01Prc -0.001232 0.000509 Inf  -2.421 0.2310 
 sexstrike ctrl vb01Prc - metoo ctrl ag01Prc      0.008914 0.000349 Inf  25.554 <.0001 
 sexstrike ctrl vb01Prc - sexstr

In [41]:
summary(as.glht(pairs(m3.0)), test=adjusted("free"))

Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > abseps”
Warning message in tmp$pfunction("adjusted", ...):
“Completion with error > 


	 Simultaneous Tests for General Linear Hypotheses

Linear Hypotheses:
                                                       Estimate Std. Error z value Pr(>|z|)    
metoo ctrl vb01Prc - sexstrike ctrl vb01Prc == 0     -0.0036452  0.0003488 -10.450  < 2e-16 ***
metoo ctrl vb01Prc - metoo orig vb01Prc == 0          0.0044540  0.0001698  26.237  < 2e-16 ***
metoo ctrl vb01Prc - sexstrike orig vb01Prc == 0     -0.0048777  0.0004045 -12.058  < 2e-16 ***
metoo ctrl vb01Prc - metoo ctrl ag01Prc == 0          0.0052691  0.0001386  38.017  < 2e-16 ***
metoo ctrl vb01Prc - sexstrike ctrl ag01Prc == 0      0.0020313  0.0003488   5.823 1.94e-08 ***
metoo ctrl vb01Prc - metoo orig ag01Prc == 0          0.0124974  0.0001698  73.618  < 2e-16 ***
metoo ctrl vb01Prc - sexstrike orig ag01Prc == 0     -0.0012476  0.0004045  -3.084  0.00407 ** 
sexstrike ctrl vb01Prc - metoo orig vb01Prc == 0      0.0080992  0.0003527  22.965  < 2e-16 ***
sexstrike ctrl vb01Prc - sexstrike orig vb01Prc == 0 -0.0012325 

In [42]:
m2.0 <- emmeans(m0,c("CAMP","TYPE"))
m2.0
pairs(m2.0)
summary(as.glht(pairs(m2.0)),test=adjusted("free"))


Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 1505108' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 1505108)' or larger];
but be warned that this may result in large computation time and memory use.

Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 1505108' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 1505108)' or larger];
but be warned that this may result in large computation time and memory use.

NOTE: Results may be misleading due to involvement in interactions



 CAMP      TYPE  emmean        SE  df asymp.LCL asymp.UCL
 metoo     ctrl 0.04349 9.091e-05 Inf   0.04331   0.04367
 sexstrike ctrl 0.04693 2.621e-04 Inf   0.04642   0.04744
 metoo     orig 0.03765 9.981e-05 Inf   0.03745   0.03784
 sexstrike orig 0.04918 3.086e-04 Inf   0.04858   0.04979

Results are averaged over the levels of: MEAS_LEVEL 
Degrees-of-freedom method: asymptotic 
Confidence level used: 0.95 

 contrast                        estimate       SE  df z.ratio p.value
 metoo ctrl - sexstrike ctrl     -0.00344 0.000277 Inf -12.405 <.0001 
 metoo ctrl - metoo orig          0.00584 0.000135 Inf  43.266 <.0001 
 metoo ctrl - sexstrike orig     -0.00570 0.000322 Inf -17.710 <.0001 
 sexstrike ctrl - metoo orig      0.00928 0.000280 Inf  33.097 <.0001 
 sexstrike ctrl - sexstrike orig -0.00226 0.000405 Inf  -5.571 <.0001 
 metoo orig - sexstrike orig     -0.01154 0.000324 Inf -35.576 <.0001 

Results are averaged over the levels of: MEAS_LEVEL 
Degrees-of-freedom method: asymptotic 
P value adjustment: tukey method for comparing a family of 4 estimates 


	 Simultaneous Tests for General Linear Hypotheses

Linear Hypotheses:
                                       Estimate Std. Error z value Pr(>|z|)    
metoo ctrl - sexstrike ctrl == 0     -0.0034415  0.0002774 -12.405  < 2e-16 ***
metoo ctrl - metoo orig == 0          0.0058412  0.0001350  43.266  < 2e-16 ***
metoo ctrl - sexstrike orig == 0     -0.0056972  0.0003217 -17.710  < 2e-16 ***
sexstrike ctrl - metoo orig == 0      0.0092827  0.0002805  33.097  < 2e-16 ***
sexstrike ctrl - sexstrike orig == 0 -0.0022557  0.0004049  -5.571 2.53e-08 ***
metoo orig - sexstrike orig == 0     -0.0115384  0.0003243 -35.576  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- free method)


# Helpers

In [43]:
?mixed